# fetch_tools

> Utilities that facilitate retrieval of data from the web

In [ ]:
#| default_exp fetch_tools

In [ ]:
#| export
from __future__ import annotations
import requests
from io import StringIO

import pandas as pd

In [ ]:
TEST_URL = 'https://www.dropbox.com/s/96xo9f1twlu3525/firmquarter_2022q1.csv?raw=1'

In [ ]:
#| export
def get_text_file_from_url (url, #Data at this url must be readable with pandas.read_csv
             nrows: int=None, #Get only the first `nrows` from the file. If None, gets the entire file
             delimiter: str=',',
             headers: dict=None, #Headers to pass to the request
             skiprows: int=None, #Skip the first `skiprows` rows
             encoding: str='utf-8', #Encoding to use when reading the file
             **pd_read_csv_kwargs,
    ) -> pd.DataFrame:
    "Gets the first `nrows` from the file found at `url`. Data at `url` must be separated by `delimiter` and be readable by pandas.read_csv"


    if nrows is None:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.content.decode(encoding)
        return pd.read_csv(StringIO(data), delimiter=delimiter, skiprows=skiprows, **pd_read_csv_kwargs)

    response = requests.get(url, stream=True, headers=headers)
    response.raise_for_status()
    if skiprows is not None: nrows += skiprows
    lines = []
    for i, line in enumerate(response.iter_lines()):
        if skiprows is not None and i < skiprows: continue
        if i >= nrows: break
        lines.append(line.decode(encoding))
    data = '\n'.join(lines)
    return pd.read_csv(StringIO(data), delimiter=delimiter, **pd_read_csv_kwargs)


In [ ]:
df = get_text_file_from_url(TEST_URL, nrows=5, delimiter='\t')
df

,gvkey,date,PRisk,NPRisk,Risk,PSentiment,NPSentiment,Sentiment,PRiskT_economic,PRiskT_environment,...,Covid_Risk,SARS_Exposure,H1N1_Exposure,Zika_Exposure,Ebola_Exposure,Brexit_Exposure,Brexit_Neg_Sentiment,Brexit_Pos_Sentiment,Brexit_Net_Sentiment,Brexit_Risk
0,1004,2002q1,359.55072,2928.6014,168.98235,997.864150,5550.5807,469.39542,9001.563,6331.43,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1004,2002q2,0.00000,0.0000,0.00000,1594.732100,-5656.6074,544.82417,0.000,0.00,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,1004,2002q3,0.00000,0.0000,0.00000,49.334494,-17818.4180,318.47134,0.000,0.00,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,1004,2003q3,0.00000,0.0000,0.00000,2581.944100,81710.4830,1314.82830,0.000,0.00,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = get_text_file_from_url(TEST_URL, nrows=5, delimiter='\t', usecols=['gvkey','date', 'PRisk'])
df

,gvkey,date,PRisk
0,1004,2002q1,359.55072
1,1004,2002q2,0.00000
2,1004,2002q3,0.00000
3,1004,2003q3,0.00000


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()